In [1]:
path = './dados/stackoverflow_perguntas.csv'

In [2]:
import pandas as pd
df = pd.read_csv(path)

In [3]:
df.shape

(5408, 2)

In [4]:
df.sample(10)

,Perguntas,Tags
294,Galera tenho o seguinte input dentro de um for...,html
466,Criei um chat com nodejs e MySQL pra buscar in...,node.js
2356,Eu tenho um botão e nele preciso usar onclick ...,html
5241,Bom dia. Existe alguma forma de quando o usuár...,html
4353,Tenho uma página com vários elementos estático...,jquery
5205,bom dia! Eu pesquisei (e até achei um js chama...,html
1929,Olá. Utilizo o flot.js para plotar um gráfico...,jquery
1067,estou testando uma função que abre pop-up para...,jquery
717,Últimamente eu tenho acessado vários sites de ...,jquery
3001,JS CODE HTML CODE O que está acontecendo...,jquery html


In [5]:
df['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [6]:
#vamos criar um modo de converter de string para inteiros binarios
# Removendo espaços em excesso nas strings
df['Tags'] = df['Tags'].str.strip()

# Separando as tags em colunas binárias
tags_dummies = df['Tags'].str.get_dummies(sep=' ')

# Adicionando as colunas resultantes ao DataFrame original
df = pd.concat([df, tags_dummies], axis=1)

In [7]:
df.sample(3)

,Perguntas,Tags,angular,html,jquery,node.js
3560,Preciso criar um bot que entra na página inici...,jquery html node.js,0,1,1,1
3169,Gostaria de colocar o menu dropdown no meu sit...,html,0,1,0,0
1903,"O problema é o seguinte, ao quebrar a linha no...",jquery html,0,1,1,0


In [8]:
df['juncao'] = list(zip(df['angular'], df['html'], df['jquery'], df['node.js']))

In [9]:
df.sample(10)

,Perguntas,Tags,angular,html,jquery,node.js,juncao
1726,Bom dia galera tenho o seguinte codigo mandand...,jquery html,0,1,1,0,"(0, 1, 1, 0)"
895,"Viva, Tenho 2 gráficos e queria trazer para el...",html,0,1,0,0,"(0, 1, 0, 0)"
2702,É possível realizarmos uma requisição do tipo ...,jquery,0,0,1,0,"(0, 0, 1, 0)"
2639,Em Java: CODE Em C++ CODE Como ler input...,node.js,0,0,0,1,"(0, 0, 0, 1)"
534,Estou terminando um HTML para me auxiliar nas ...,html,0,1,0,0,"(0, 1, 0, 0)"
855,Toda vez que eu tento fazer alguma modificação...,node.js,0,0,0,1,"(0, 0, 0, 1)"
301,Tenho o seguinte input file que faz o upload d...,html angular,1,1,0,0,"(1, 1, 0, 0)"
4395,Estou desenvolvendo uma aplicação híbrida para...,jquery,0,0,1,0,"(0, 0, 1, 0)"
1221,Eu tenho a seguinte logica de autenticação CO...,angular,1,0,0,0,"(1, 0, 0, 0)"
3539,Estou desenvolvendo um sistema web onde precis...,node.js,0,0,0,1,"(0, 0, 0, 1)"


In [10]:
# quais classificadores usar para multilabel

In [11]:
# hora do modelo

In [12]:
from sklearn.model_selection import train_test_split
X = df['Perguntas']
y = df['juncao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# TF-IDF
# texto para numeros - A principal característica do TF-IDF é ser uma pontuação proporcional à frequência da palavra
# no texto e equilibrada pela frequência no corpus, ou seja, palavras que se repetem muito tendem a ter pontuações 
# menores e são menos relevantes no processo de classificação.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000,max_df=0.85) # maximo  vetor de palavras e remover palavras com muita frequencia

In [15]:
vectorizer.fit(df.Perguntas)

TfidfVectorizer(max_df=0.85, max_features=5000)

In [16]:
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'único', 'únicos', 'útil'], dtype=object)

In [17]:
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf  = vectorizer.transform(X_test)

In [18]:
# relevancia binaria onevsrestclassifier

In [19]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
classificador = OneVsRestClassifier(lg)

In [20]:
classificador

OneVsRestClassifier(estimator=LogisticRegression())

In [21]:
import numpy as np 
y_train_array = np.asarray(list(y_train))
y_test_array = np.asarray(list(y_test))

In [22]:
classificador.fit(X_train_tfidf, y_train_array)

OneVsRestClassifier(estimator=LogisticRegression())

In [23]:
classificador.score(X_test_tfidf,y_test_array)

0.3927170868347339

In [24]:
scr = classificador.score(X_test_tfidf,y_test_array)

In [25]:
# hamming loss para multilabel
# A principal diferença é que o Hamming loss avalia os acertos de cada label individualmente, 
# enquanto a acurácia avalia a combinação das labels, ou seja, será considerado correto apenas se o 
# classificador acertar todas as labels de uma determinada instância. Para o Hamming Loss, 
# quanto mais próximo de 0 melhor o resultado, enquanto para a acurácia o melhor resultado é o mais próximo de 1.

In [26]:
from sklearn.metrics import hamming_loss

previsao_onevsrest = classificador.predict(X_test_tfidf)
hamming_loss_onevsrest = hamming_loss(y_test_array, previsao_onevsrest)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

Hamming Loss  0.19


In [32]:
# Hamming Loss, quando mais proximo de 0 melhor.

In [30]:
from skmultilearn.problem_transform import ClassifierChain
cl_multi = ClassifierChain(lg)

In [31]:
cl_multi.fit(X_train_tfidf, y_train_array)
scr = cl_multi.score(X_test_tfidf,y_test_array)
scr

0.492436974789916

In [33]:
#49%

In [34]:
from skmultilearn.problem_transform import BinaryRelevance

In [35]:
cl_multi_bin = BinaryRelevance(lg)
cl_multi_bin.fit(X_train_tfidf, y_train_array)
scr_b = cl_multi_bin.score(X_test_tfidf,y_test_array)
scr_b

0.3927170868347339

In [39]:
from skmultilearn.adapt import MLkNN

In [44]:
cl_multi_mlknn = MLkNN(k=3)
cl_multi_mlknn.fit(X_train_tfidf, y_train_array)
scr_mlknn = cl_multi_mlknn.score(X_test_tfidf,y_test_array)
scr_mlknn

TypeError: NearestNeighbors.__init__() takes 1 positional argument but 2 were given